### Install the package

In [1]:
#!pip install git+https://github.com/gabri14el/image-segmentation-keras
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


### Download the dataset

In [2]:
import tensorflow as tf
import os
save_path = r'.\\' ''
experiment_description = 'experimento4'

#diretorio onde está o dataset
out = r'C:\Users\Gabriel\Downloads\Saida_'+experiment_description

main_dir = r'C:\Users\Gabriel\OneDrive - Universidade de Tras-os-Montes e Alto Douro\UTAD\2020-2021\Pesquisa\Dataset\teste_masks_crop3'

#subpastas que diferenciam imagem de mascaras
image_dir = 'image'
mask_dir = 'mask'

# set the necessary directories
trainimg_dir = os.path.join(main_dir, 'train', image_dir)
trainmsk_dir = os.path.join(main_dir, 'train', mask_dir)
valimg_dir = os.path.join(main_dir, 'validation', image_dir)
valmsk_dir = os.path.join(main_dir, 'validation', mask_dir)
testimg_dir = os.path.join(main_dir, 'test', image_dir)
testmsk_dir = os.path.join(main_dir, 'test', mask_dir)

batch = 6
LR = 0.0001
EPOCHS = 50

train_steps = len(os.listdir(trainimg_dir))/batch
val_steps = len(os.listdir(valimg_dir))/batch

target_size = (320, 320)

### Initialize the model

In [3]:
from keras_segmentation.models.segnet import resnet50_segnet

model = resnet50_segnet(n_classes=2 ,  input_height=target_size[0], input_width=target_size[1], encoder_level=3, n_up=3)


##Define callbacks

In [4]:
#os.makedirs(save_path, exist_ok = True)

path_weight = os.path.join(save_path, experiment_description+'-weights.h5')
path_model = os.path.join(save_path, experiment_description+'-model')

model_checkpoint_weights_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=path_weight,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_checkpoint_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=path_model,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1_m', factor=0.95, patience=2, verbose=1, mode='max', cooldown=1, min_lr=0.000001),

callbacks = [model_checkpoint_weights_callback, model_checkpoint_model_callback, reduce_on_plateau]


### Train the model

In [5]:
import segmentation_models as sm
n_classes = 6
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

with tf.device('/gpu:0'):
    model.train(
        train_images =  trainimg_dir,
        train_annotations = trainmsk_dir,
        batch_size=batch,
        val_images=valimg_dir,
        val_annotations=valmsk_dir,
        val_batch_size=batch,
        checkpoints_path = "segnet",
        callbacks = callbacks,
        epochs=EPOCHS,
        verify_dataset=None,
        steps_per_epoch=train_steps,
        val_steps_per_epoch=val_steps,
        validate = True,
        fine_tuning = True,
        layer_fine_tuning = 'bn4f_branch2c',
        learning_rate = LR,
        loss = total_loss
    )

Segmentation Models: using `tf.keras` framework.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 320, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 326, 326, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 160, 160, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 160, 160, 64) 256         conv1[0][0]                      
___________________________________________

C:\Users\Gabriel\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: .\experimento4-model\assets
Epoch 2/50
702/883 [======================>.......] - ETA: 38s - loss: 0.0992 - accuracy: 0.9368 - mean_io_u: 0.2502 - f1_m: 0.9368

# Model Evaluation

In [ ]:
model.load_weights(path_weight)

from keras_segmentation.predict import evaluate

out = evaluate(model,
    inp_images_dir= testimg_dir,
    annotations_dir = testmsk_dir
)

print(out)

2727it [05:41,  7.98it/s]

{'frequency_weighted_IU': 0.9043723983835286, 'mean_IU': 0.7670220757970758, 'class_wise_IU': array([0.58996104, 0.94408311])}
